# Interaktiva grafer för icke-ideala vätskeblandningar

Välkommen till en samling interaktiva grafer som illustrerar ideala och icke-ideala vätskeblandningar på kursen Termodynamik och ytkemi (KFKA10).

## Instruktioner

För att starta alla tre graferna, välj <i>Run</i> -> <i>Run all cells</i> i menyn.

Gemensamt för alla exemplen är att de använder Bragg-Williams-modellen, i vilken avvikelsen från ideal lösning beskrivs av en enda parameter &chi; (chi). När &chi; är större än 0 trivs molekylerna A och B <i>sämre</i> ihop än i en ideal lösning. När &chi; är mindre än 0 trivs molekylerna A och B <i>bättre</i> ihop än i en ideal lösning.

### Graf 1: Ångtryck som funktion av sammansättningen.

Förutom &chi; kan du variera ångtrycken för de rena ämnena, pA* och pB*. Om &chi;=0 erhålls förstås det vanliga tillståndsdiagrammet för ideal lösning (Raoults lag). Om &chi;>0 erhålls en positiv avvikelse från Raoults lag, alltså att &gamma;>1. Om &chi;<0 erhålls en negativ avvikelse från Raoults lag, alltså att &gamma;<1.

### Graf 2: Gibbs blandningsenergi (ändringen när man tar bort en skiljevägg mellan två vätskor med samma tryck och temperatur)

Här kan du bara variera &chi; och se hur &Delta;H och &Delta;G påverkas. Du kan också se minimat i Gibbs energi när man blandar lika mängd av vätskorna. När molekylerna trivs riktigt dåligt tillsammans (&chi; > 2) så kan du se att det blir två minima, som inte ligger vid xA=0.5. Det är alltså då mer fördelaktigt för blandningen att separera i två faser med olika sammansättning (tänk olja och vatten som ett extremt exempel).

### Graf 3: Kokpunktsdiagram

Här kan du förutom &chi; variera de normala kokpunkterna (TA och TB, i Kelvin) och ångbildningsentalpierna (HA och HB, i kJ/mol) för de båda ämnena. I grafen kan du se hur kokpunktskurvan och kondensationspunktskurvan ändras. Försök ställa in parametrarna så att du får en azeotrop!

I koden kan du se att ångtrycken för de rena ämnena A och B vid en viss temperatur beräknas med Clausius-Clapeyrons ekvation, och att vi sedan bestämmer kokpunkten för varje sammansättning genom att lösa ekvationen pA+pB = 1 atm. Genom att även plotta ångans sammansättning så erhålls även ånglinjen (kondensationspunkten).


## Bragg-Williams-modellen (för den intresserade)

Modellen är en så kallad gitter-modell där molekylerna antas ligga i ett statiskt tredimensionellt gitter ("rutnät"), där varje plats innehåller en molekyl, antingen A eller B. Varje molekyl har <i>z</i> grannar och interaktionsenergin mellan två grannar antas vara wAA om båda är A-molekyler, wBB om båda är B-molekyler och wAB om det är en A och en B. Alla interaktionsenergier mellan molekyler som inte är grannar antas vara 0. Interaktionsparametern &chi; definieras som

RT&chi;=z(wAB - 1/2 wAA - 1/2 wBB)

Modellen antar sedan att A- och B-molekylerna är slumpvis fördelade i gittret, vilket naturligtvis är en approximation eftersom det i verkligheten skulle ligga färre B-molekyler runt en A-molekyl än det genomsnittliga värdet om &chi;>0 och tvärtom om &chi;<0.

In [1]:
from numpy import *
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy
import scipy.optimize
from scipy.optimize import fsolve

plt.rcParams['figure.figsize'] = [12,8]
plt.rcParams['figure.dpi'] = 100

atm=1.01325E5
torr=atm/760.0
R=8.3145
g=9.807
NA=6.022e23
def tok(x): return x+273.15



RT=2.5 #kJ/mol
#def slog(x): return log(x) if x>0 else 0   #DOES NOT WORK
def pB(xB,chi=0,pBs=10): return xB*exp(chi*(1-xB)**2)*pBs
def pA(xB,chi=0,pAs=20): return (1-xB)*exp(chi*xB**2)*pAs
def DF(xB,chi=0): return RT*(chi*xB*(1-xB)+xB*log(xB)+(1-xB)*log(1-xB))
def DU(xB,chi=0): return RT*(chi*xB*(1-xB))
def TDS(xB,chi=0): return RT*(xB*log(xB)+(1-xB)*log(1-xB))  #-TDS
    
xB=linspace(0.001,0.999)

In [2]:
def update(chi,pAs,pBs):
    plt.figure(2)
    plt.xlabel('xB')
    plt.ylabel('Ångtryck')
    plt.ylim(0,100)
    plt.xlim(0,1)
    plt.plot(xB,pA(xB,0,pAs),'r--', label="pA (ideal)")
    plt.plot(xB,pB(xB,0,pBs),'b--',label="pB (ideal)")
    plt.plot(xB,pA(xB,0,pAs)+pB(xB,0,pBs),'k--',label="ptot (ideal)")
    plt.plot(xB,pA(xB,chi,pAs),'r-', label="pA")
    plt.plot(xB,pB(xB,chi,pBs),'b-', label="pB")
    plt.plot(xB,pA(xB,chi,pAs)+pB(xB,chi,pBs),'k-', label="ptot")
    plt.legend()
    plt.show()
    
widgets.interact(update, chi = widgets.FloatSlider(value=0,
                                               min=-2.0,
                                               max=4.0,
                                               step=0.25),
                pAs = widgets.FloatSlider(value=20,
                                               min=0,
                                               max=100,
                                               step=1),
                pBs = widgets.FloatSlider(value=10,
                                               min=0,
                                               max=100,
                                               step=1))

interactive(children=(FloatSlider(value=0.0, description='chi', max=4.0, min=-2.0, step=0.25), FloatSlider(val…

<function __main__.update(chi, pAs, pBs)>

In [3]:
def update(chi):
    fig = plt.figure(1)
    plt.xlabel('xB')
    plt.ylabel('Gibbs energi (kJ/mol)')
    plt.ylim(-3,3)
    plt.xlim(0,1)

    plt.plot(xB,DU(xB,chi),'r-', label=r'$\Delta H$')
    plt.plot(xB,TDS(xB,chi),'b-',label=r'$-T\Delta S$')
    plt.plot(xB,DF(xB,chi),'k-',label=r'$\Delta G$')
    plt.plot(xB,0*xB,'g--')
    plt.legend()

    if(chi>2):
        x1,=fsolve(lambda x: log(x/(1-x))+chi*(1-2*x), 0.01)
        x2=1-x1
    else:
        x1=x2=0.5
    plt.plot(x1,DF(x1,chi),'go')
    plt.plot(x2,DF(x2,chi),'go')
    plt.show()

    
widgets.interact(update, chi = widgets.FloatSlider(value=0,
                                               min=-2.0,
                                               max=4.0,
                                               step=0.2))


interactive(children=(FloatSlider(value=0.0, description='chi', max=4.0, min=-2.0, step=0.2), Output()), _dom_…

<function __main__.update(chi)>

In [6]:
points=set()

def update(T):
    if(T>350) or (T>300 and T%4==0) or (T>300 and T%6==0) or (T>250 and T%10==0) or (T%20==0):
        pass
    else:
        return
    chi=800/T
    fig = plt.figure(1)
    plt.xlabel('xB')
    plt.ylabel('Temperature (K)')
    plt.ylim(50,450)
    plt.xlim(0,1)

    plt.plot(xB,150+100*DF(xB,chi),'k-',label=r'$\Delta G$ (arbitrary scale)')
    plt.plot(xB,150+0*xB,'g--')
    

    if(chi>2):
        x1,=fsolve(lambda x: log(x/(1-x))+chi*(1-2*x), 0.01)
        x2=1-x1
    else:
        x1=x2=0.5



    plt.plot(x1,150+100*DF(x1,chi),'go')
    plt.plot(x2,150+100*DF(x2,chi),'go')

    if(chi>=2):
        points.add((x1,T))
        points.add((x2,T))


    for (x,TT) in points:
        plt.plot(x,TT,'bo')
        
    if(chi>=2):
        plt.plot(x1,T,'ro', label='Free energy minima')
        plt.plot(x2,T,'ro')

        
    plt.plot(xB,T+0*xB,'b-')

    plt.legend()
    plt.show()
    

    
widgets.interact(update, T = widgets.FloatSlider(value=200,
                                               min=200,
                                               max=450,
                                               step=2))



interactive(children=(FloatSlider(value=200.0, description='T', max=450.0, min=200.0, step=2.0), Output()), _d…

<function __main__.update(T)>

In [3]:

x=linspace(0.001,0.999)

def update(chi,TA,TB,HA,HB):
    def pAs(T): return atm*exp(HA*1e3/R*(1/TA-1/T))
    def pBs(T): return atm*exp(HB*1e3/R*(1/TB-1/T))
    def pA(T,xA): return xA*exp(chi*(1-xA)**2)*pAs(T)
    def pB(T,xA): return (1-xA)*exp(chi*xA**2)*pBs(T)
    def ptot(T,xA): return pA(T,xA)+pB(T,xA)

    fig = plt.figure()
    plt.xlabel(r'$x_A$')
    plt.ylabel('Temperatur (K)')
    plt.ylim(320,380)
    plt.xlim(0,1)
    
    Tblist=[]
    yAlist=[]
    for xA in x:
        Tb,=fsolve(lambda T: ptot(T,xA)-atm, TA)
        yA=pA(Tb,xA)/ptot(Tb,xA)
        Tblist.append(Tb)
        yAlist.append(yA)
    
    plt.plot(x,Tblist,'r-', label='kokpunkt')
    plt.plot(yAlist,Tblist,'b-', label='kondensationspunkt')
    plt.legend()
    plt.show()

widgets.interact(update, chi = widgets.FloatSlider(value=0,
                                               min=-2.0,
                                               max=2.0,
                                               step=0.1),
                TA = widgets.FloatSlider(value=370,
                                               min=330,
                                               max=370,
                                               step=1),
                TB = widgets.FloatSlider(value=350,
                                               min=330,
                                               max=370,
                                               step=1),
                HA = widgets.FloatSlider(value=50,
                                               min=20,
                                               max=60,
                                               step=1),
                HB = widgets.FloatSlider(value=50,
                                               min=20,
                                               max=60,
                                               step=1));





interactive(children=(FloatSlider(value=0.0, description='chi', max=2.0, min=-2.0), FloatSlider(value=370.0, d…